In [ ]:
#using Pkg

#Pkg.activate("/Users/tims/.julia/dev/FUSE_Design")
using Revise
using FUSE

using Plots;
FUSE.logging(Logging.Info; actors=Logging.Error);


In [ ]:
# Simple power plant optimization (not realistic as it's missing constraints but it will get you started)

ini,act = FUSE.case_parameters(:KDEMO);

### Act settings
act.ActorEquilibrium.model = :TEQUILA
act.ActorCoreTransport.model = :FluxMatcher

act.ActorPFdesign.model=:optimal
act.ActorFluxSwing.operate_oh_at_j_crit = true # this maximizes flattop inside the fluxswing actor

act.ActorWholeFacility.update_plasma = true

# This is handeled by the constraint funcitons
act.ActorHFSsizing.error_on_performance = false
act.ActorHFSsizing.error_on_technology = false

act.ActorCoreTransport.model = :FluxMatcher
act.ActorFluxMatcher.evolve_densities = :flux_match
act.ActorFluxMatcher.max_iterations = 500

ini.equilibrium.B0 = ini.equilibrium.B0 ↔ [3.0, 15.0]
ini.equilibrium.ip = ini.equilibrium.ip ↔ [4.0e6, 22e6]
ini.equilibrium.R0 = ini.equilibrium.R0 ↔ [4.0, 8.0]
ini.equilibrium.pressure_core = missing

act.ActorPedestal.model = :EPED
act.ActorFluxMatcher.rho_transport = 0.2:0.05:0.8
act.ActorTGLF.tglfnn_model ="sat0quench_em_d3d+mastu_azf+1"
act.ActorFluxMatcher.algorithm = :simple
act.ActorFluxMatcher.evolve_pedestal = true

ini.requirements.lh_power_threshold_fraction = 1.

# Requirements

ini.requirements.flattop_duration = missing
ini.requirements.log10_flattop_duration = missing # log10 versionis not needed

ini.requirements.power_electric_net = 50e6 # 50 +/- 10 MWe
ini.requirements.tritium_breeding_ratio = 1.1

ini.requirements.coil_j_margin = 0.1
ini.requirements.coil_stress_margin = 0.1

IMAS.update_ObjectiveFunctionsLibrary!()
IMAS.update_ConstraintFunctionsLibrary!()

OFL = deepcopy(IMAS.ObjectiveFunctionsLibrary)
CFL = deepcopy(IMAS.ConstraintFunctionsLibrary)

objective_functions = [OFL[:min_capital_cost]]

constraint_functions = [
        CFL[:power_electric_net],
        CFL[:min_lh_power_threshold_fraction],
        CFL[:max_tf_coil_j],CFL[:max_oh_coil_j],
        CFL[:max_pl_stress],CFL[:max_tf_coil_stress],CFL[:max_oh_coil_stress]]
end

println("== OBJECTIVE FUNCTIONS ==")
display(objective_functions)
println()
println("== CONSTRAINT FUNCTIONS ==")
display(constraint_functions)

In [ ]:
# sty is the act equivalent for a study, it has common parameters like server and n_workers but also study dependent parameters like n_simulations
sty = FUSE.study_parameters(:MultiObjectiveOptimizer);
typeof(sty)

In [ ]:
# Interacting with sty
sty.server = "localhost" # this can be set to saga/omega/your_cluster
sty.n_workers = 2

dirr = joinpath(pwd(),"test_moop") # feel free to change this directory as you probably don't want to store it here
if !isdir(dirr)
    sty.save_folder = mkdir(dirr)
else
    sty.save_folder = dirr
end

sty.restart_workers_after_n_generations = 5 # this is the default behavior and releases workers after running the study

sty.population_size = 20
sty.number_of_generations = 5

# For a reaslistic study you need about population_size = 300 and number_of_generations = 100

sty

In [ ]:
# study is the actor equivalent of actors, here sty, act and outputs are kept of the workflow
study = FUSE.StudyMultiObjectiveOptimizer(sty,ini, act, constraint_functions, objective_functions); # it is possible to pass in keyword arguments to sty
# instantiating the study will also setup the study i.e. FUSE.setup(study);

# for now you will have to import FUSE everywhere in your distributed computing
using Distributed
@everywhere import FUSE
@everywhere import IJulia

In [ ]:
Distributed.workers()

In [ ]:
FUSE.run(study);   # runs the study

# at the end of the run workers are released so you will have to setup again if you want to run more